In [2]:
!pip install -q accelerate -U
!pip install -q bitsandbytes -U
!pip install -q trl -U
!pip install -q peft -U
!pip install -q transformers -U
!pip install datasets -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 2.5 MB/s eta 0:00:00


In [52]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/ultrachat_200k", trust_remote_code=True, split='train_sft')

In [53]:
dataset

Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 207865
})

In [54]:
dataset = dataset.shuffle(seed=0).select(range(10_000))

In [55]:
from transformers import AutoTokenizer

template_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

def format_prompt(example):
    ''' Format the prompt using the <|user|> and <|assistant|> format'''
    chat = example['messages']
    prompt = template_tokenizer.apply_chat_template(chat, tokenizer=False)

    return {'text':str(prompt)}

In [56]:
print(format_prompt(dataset[0]).keys())
print(format_prompt(dataset[0])['text'])

dict_keys(['text'])
[529, 29989, 1792, 29989, 29958, 13, 29941, 29889, 940, 271, 278, 18655, 1849, 297, 1045, 6504, 4094, 363, 278, 13622, 931, 29889, 2, 29871, 13, 29966, 29989, 465, 22137, 29989, 29958, 13, 29902, 437, 451, 505, 2472, 1048, 278, 2702, 1134, 310, 18655, 1849, 1641, 12992, 304, 29889, 2398, 29892, 1244, 526, 2498, 11994, 363, 1045, 6504, 1556, 18655, 1849, 29901, 13, 13, 29896, 29889, 399, 1161, 278, 18655, 1849, 26606, 411, 5941, 4094, 29889, 13, 29906, 29889, 315, 329, 278, 18655, 1849, 964, 2319, 470, 18350, 29899, 29879, 1891, 12785, 29889, 13, 29941, 29889, 1771, 292, 263, 3104, 310, 4094, 304, 1045, 309, 373, 278, 380, 994, 29889, 13, 29946, 29889, 3462, 263, 12534, 305, 310, 15795, 304, 278, 1045, 6504, 4094, 29889, 13, 29945, 29889, 3462, 278, 18655, 1849, 304, 278, 1045, 6504, 4094, 29889, 13, 29953, 29889, 17278, 278, 18655, 1849, 363, 278, 13622, 931, 313, 4149, 7984, 292, 11994, 373, 278, 3577, 470, 1106, 701, 7984, 292, 3064, 363, 2702, 18655, 1849, 7395, 

In [57]:
dataset = dataset.map(format_prompt)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2724 > 2048). Running this sequence through the model will result in indexing errors


## Testing Base LLAMA Model


*   Let's see how base Tiny-LLAMA performs out of the box



In [58]:
from transformers import pipeline

# base model
model_name ='TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T'

pipe = pipeline(task='text-generation', model=model_name, device='cuda')

# prompt
# <|user>, <|assistant|>

prompt = '''<|user|>
Tell me something about LLM's.
<|assistant|>
'''

prompt = '''
Tell me something about LLM's.
'''

output = pipe(prompt)
output

[{'generated_text': '\nTell me something about LLM\'s.\n\nA: I think you are looking for the word "tell" in the context of "something about LLM\'s".\n\nA: I think you are looking for the word "tell" in the context of "something about LLM\'s".\n'}]

## Model configuration for training

In [59]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [60]:
# do the 4-bit quantization configuration in Q-LoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_comput_dtype='float16',
    bnb_4bit_use_double_quant=True
)

Unused kwargs: ['bnb_4bit_comput_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [61]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token='<PAD>'
tokenizer.padding_size='left'

In [62]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = 'auto',
    quantization_config=bnb_config
)

In [63]:
model.config.use_cache=False
model.config.pretraining_tp =1

In [64]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05

## Prepare LoRA Configuration for PEFT Fine tuning

In [65]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj']
)

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, peft_config)

In [66]:
w = 2048*256
a = 2048*64
b = 64*256

w, a, b, a+b, (a+b)/w

(524288, 131072, 16384, 147456, 0.28125)

In [67]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

In [68]:
dataset

Dataset({
    features: ['prompt', 'prompt_id', 'messages', 'text'],
    num_rows: 10000
})

## Model Fine Tuning

In [ ]:
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer

output_dir = 'train_dir'

args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim='paged_adamw_32bit',
    learning_rate=2e-4,
    lr_scheduler_type='cosine',
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)

trainer = SFTTrainer(model=model,
    train_dataset=dataset,
    dataset_text_field='text',
    tokenizer=tokenizer,
    args=args,
    max_seq_length=512,
    peft_config=peft_config
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.350600
20,1.187300
30,1.150500
40,1.131400
50,1.098500
60,1.067000
70,1.067000
80,1.047200
90,1.066400
100,1.036700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


In [ ]:
trainer.model.save_pretrained("TinyLlama-1.1B-qlora")

## Load Pre-Trained PEFT Model for Prediction

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    device_map='auto'
)

merged_model = model.merge_and_unload()

In [ ]:
from transformers import pipeline

prompt='''<|User|>
Tell me something about Large Language Models.</s>
<|assistant|>
'''
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = '<PAD>'
tokenizer.padding_size='left'

pipe = pipeline(task='text-generation', model=merged_model, tokenizer=tokenizer)
output = pipe(prompt)
print(output[0]['generated_text'])

In [ ]:
## output:
'''<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
Large Language Models (LLMs) are a type of artificial intelligence (AI) that can generate human-like language. They are trained on large amounts of text data, such as Wikipedia articles, to create a model that can generate natural-sounding sentences.

LLMs are becoming increasingly popular in various fields, including natural language processing (NLP), machine translation, and chatbots. They are used to generate text in a variety of languages, including English, French, German, and Spanish.

One of the key advantages of LLMs is their ability to generate natural-sounding sentences that are similar to human language. This is achieved through the use of deep learning algorithms that learn to recognize patterns in text data and generate sentences that are similar to human language.

Another advantage of LLMs is their ability to handle complex language tasks, such as machine translation, where human translators struggle to generate accurate translations. LLMs can generate natural-sounding sentences that are accurate and comprehensible, even when dealing with difficult language combinations.

Overall, LLMs are a powerful tool that can help to improve the quality and efficiency of various NLP applications. They are becoming increasingly popular in a variety of fields, and their use is expected to continue to grow in the futur
'''

In [ ]:
!zip -r tiny_llama_qlora_adapter.zip TinyLlama-1.1B-qlora